# ¡Tu primer proyecto con LLM en Python!

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aldomunaretto/immune_generative_ai/blob/main/notebooks/01_primer_proyecto_llm.ipynb)

### Importamos las librerias necesarias

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Conexión a OpenAI

La siguiente celda es donde cargamos las variables de entorno en su archivo `.env` y nos conectamos a OpenAI.

### Cargamos las variables del fichero .env

In [ ]:
load_dotenv()

### Verificamos que la clave de API de OpenAI esté configurada correctamente

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No se encontró ninguna clave API: diríjase al cuaderno de resolución de problemas en esta carpeta para identificarla y solucionarla.")
elif not api_key.startswith("sk-proj-"):
    print("Se encontró una clave API, pero no inicia sk-proj-; verifique que esté usando la clave correcta; consulte el cuaderno de resolución de problemas")
elif api_key.strip() != api_key:
    print("Se encontró una clave API, pero parece que puede tener espacios o caracteres de tabulación al principio o al final; elimínelos; consulte el cuaderno de resolución de problemas")
else:
    print("¡Se encontró la clave API y hasta ahora parece buena!")




### Ahora realizamos la conexión con la API de OpenAI.

In [ ]:
openai = OpenAI()

en caso de no tener configuras las varibales de entorno o no tener el fichero .env lo podemos hacer de la seiguente forma:

~~~python
openai = OpenAI(api_key="your-key-here-starting-sk-proj-")
~~~

## Clase Website para scrappear sitios web

In [ ]:
class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

### Vamos a probar

In [ ]:
marca = Website("https://marca.com")
print(marca.title)
print(marca.text)

## Tipos de indicaciones/prompts

Quizás ya sepas esto, pero si no, te resultará muy familiar.

Los modelos como GPT4o han sido entrenados para recibir instrucciones de una manera particular.

Esperan recibir:

- **Una indicación del sistema** que les indique qué tarea están realizando y qué tono deben usar

- **Una indicación del usuario**: el inicio de la conversación al que deben responder

### Definimos nuestro mensaje de sistema:

In [ ]:
system_prompt = """ Eres un asistente que analiza el contenido de un sitio web
y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación.
Responder en Markdown en español."""

### Definimos una función que escribe un mensaje de usuario que solicita resúmenes de sitios web:

~~~python
def user_prompt_for(website):
    user_prompt = f"""
    Estás viendo un sitio web titulado {website.title}. 
    El contenido de este sitio web es el siguiente: {website.text}.
    Si incluye noticias, productos o anuncios, resúmelos también.
    """
    return user_prompt
~~~~

#### como no todos los sitios tienen este formato podriamos la clase Website devolveria una estructura que haria necesesario cambiar esta función. Podriamos proponer esta:

~~~python
def user_prompt_for(website):
    user_prompt = f"""
    Estás viendo un sitio web titulado {website["title"]}. 
    El contenido de este sitio web es el siguiente: {website["content"]}.
    Si incluye noticias, productos o anuncios, resúmelos también.
    """
    return user_prompt
~~~

### Finalmente definimos una que tome todo en cuenta:

In [ ]:
def user_prompt_for(website):
    title = getattr(website, "title", None)
    if title is None and isinstance(website, dict):
        title = website.get("title", "")
    text = getattr(website, "text", None)
    if text is None and isinstance(website, dict):
        text = website.get("content", "")
    user_prompt = f"""
    Estás viendo un sitio web titulado {title}.
    El contenido de este sitio web es el siguiente: {text}.
    Si incluye noticias, productos o anuncios, resúmelos también.
    """
    return user_prompt

In [ ]:
print(user_prompt_for(marca))

## Mensajes

La API de OpenAI espera recibir mensajes en una estructura particular.
Muchas de las otras API comparten esta estructura:

```
[
    {"role": "system", "content": "el mensaje de sistema va aquí"},
    {"role": "user", "content": "el mensaje de usuario va aquí"}
]

In [ ]:
# Puedes ver cómo esta función crea exactamente el formato anterior

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
messages_for(marca)

## Es hora de unirlo todo: ¡la API para OpenAI es muy simple!

In [ ]:
# Y ahora: llama a la API de OpenAI. ¡Te resultará muy familiar!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://marca.com")

### Una función para mostrar esto de forma clara en la salida de Jupyter, usando markdown

In [ ]:

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://marca.com")

# Probemos con más sitios web

Ten en cuenta que esto solo funcionará en sitios web que se puedan extraer con este enfoque simplista.

Los sitios web que se renderizan con Javascript, como las aplicaciones React, no se mostrarán. Resolveremso esto más adelante con una implementación de Selenium que solucione este problema. 

Además, los sitios web protegidos con CloudFront (y similares) pueden generar errores 403.

¡Pero muchos sitios web funcionarán perfectamente!

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://anthropic.com")

In [ ]:
display_summary("https://openai.com")

### Para aquellos casos donde la página esta diseñada utilizando JavaScript o algunos de sus framework
Primero descarga el controlador (driver) de chorme desde la siguiente dirección: https://googlechromelabs.github.io/chrome-for-testing/

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service("chromedriver-mac-arm64/chromedriver")
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [ ]:
display_summary("https://anthropic.com")

In [ ]:
display_summary("https://openai.com")

### Creamos una Clase que utiliza Selenium para cargar y extraer el contenido de páginas web dinámicas.

In [ ]:
class DynamicWebsiteScraper:
    def __init__(self, driver_path):
        try:
            print("🔄 Creando instancia de DynamicWebsiteScraper...")
            options = Options()
            options.add_argument("--disable-gpu")
            options.add_argument("--no-sandbox")
            options.add_argument("--disable-dev-shm-usage")
            self.driver_service = Service(driver_path)
            self.driver = webdriver.Chrome(service=self.driver_service, options=options)
            print("✅ DynamicWebsiteScraper inicializado correctamente.\n")
        except Exception as e:
            print(f"❌ Error al inicializar DynamicWebsiteScraper: {e}")

    def scrape(self, url, wait_time=5):
        try:
            self.driver.get(url)
            time.sleep(wait_time)
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            title = soup.title.string if soup.title else "No tiene título"
            for tag in soup(["script", "style", "img", "input"]):
                tag.decompose()
            content = soup.get_text(separator="\n", strip=True)
            return {"title": title, "content": content}
        except Exception as e:
            print(f"❌ Error durante el scraping: {e}")
            return {"title": "Error", "content": ""}

    def close(self):
        try:
            self.driver.quit()
            print("✅ Navegador cerrado correctamente.\n")
        except Exception as e:
            print(f"❌ Error al cerrar el navegador: {e}")

In [ ]:
scraper = DynamicWebsiteScraper("chromedriver-mac-arm64/chromedriver")

### Volvemos a crear la función para generar un resumen

In [ ]:
def summarize(url, scraper):
    """
    Realiza el scraping del sitio web y utiliza la API de OpenAI para generar un resumen.
    :param url: URL del sitio web
    :param scraper: Instancia de DynamicWebsiteScraper
    :return: Resumen en Markdown
    """
    try:
        print(f"🔄 Realizando scraping en la URL: {url}")
        website = scraper.scrape(url)  # Usamos el scraper para obtener el contenido dinámico
        print(website)
        print(f"✅ Scraping completado con éxito en la URL: {url}\n")

        print("🔄 Generando el resumen a través de la API de OpenAI...")
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Modelo a usar
            messages=messages_for(website)  # Mensajes generados a partir del contenido scrapeado
        )
        print("✅ Resumen generado correctamente.\n")
        return response.choices[0].message.content
    except Exception as e:
        print(f"❌ Error al generar el resumen: {e}")
        return "Error al generar el resumen."

### Rehacemos la función que genera y muestra el resumen de un sitio web en formato Markdown utilizando un scraper:

In [ ]:
def display_summary(url, scraper):
    try:
        print(f"🔄 Iniciando el proceso para generar el resumen del sitio web: {url}")
        summary = summarize(url, scraper)  # Llamada a summarize con url y scraper
        
        if summary.startswith("Error"):
            print(f"❌ No se pudo generar el resumen para: {url}")
        else:
            print(f"✅ Resumen generado correctamente para: {url}\n")
            display(Markdown(summary))  # Mostrar el resumen en Markdown
    except Exception as e:
        print(f"❌ Ocurrió un error durante la visualización del resumen: {e}")

In [ ]:
display_summary("https://anthropic.com", scraper)

In [ ]:
display_summary("https://openai.com", scraper)